In [17]:
# add answers data
# Import Modules
import pandas as pd
import numpy as np
import sqlite3
import pickle
import os
import matplotlib.pyplot as plt

# Import Dataset
conn = sqlite3.connect('/data1/StackOverflow/stackexchange-to-sqlite/stack.db')
query = '''
SELECT id, creation_date, owner_user_id, body
FROM answers
WHERE creation_date > '2021-09-01'
AND creation_date < '2023-09-01';
'''
ans_df = pd.read_sql_query(query, conn)
conn.close()

# Import Users Data
conn = sqlite3.connect('/data1/StackOverflow/stackexchange-to-sqlite/stack.db')
query = '''
SELECT id, creation_date
FROM users;
'''
user_df = pd.read_sql_query(query, conn)
conn.close()
user_df = user_df.rename(columns = {'creation_date':'account_creation_date',
                                    'id': 'owner_user_id'})
ans_df = ans_df.dropna(subset = 'owner_user_id')
ans_df['owner_user_id'] = ans_df['owner_user_id'].astype(int).astype(str)
# Merge answers to the correspond users
user_df['owner_user_id'] = user_df['owner_user_id'].astype(str)
ans_df = pd.merge(ans_df, user_df, on = 'owner_user_id', how = 'left')
ans_df

,id,creation_date,owner_user_id,body,account_creation_date
0,69006420,2021-09-01 00:00:18.070,7332046,"<p>The short approach, remove python3.9 from y...",2016-12-22 19:33:51.100
1,69006421,2021-09-01 00:00:28.823,14471093,"<p>open the terminal, run as administrator.</p...",2020-10-18 06:29:51.557
2,69006422,2021-09-01 00:00:35.537,523612,"<p>Conceptually, <code>Base</code> has no reas...",2010-11-29 07:44:17.673
3,69006429,2021-09-01 00:01:09.140,11269158,<p><code>calldata</code> is a special data loc...,2019-03-27 23:15:22.047
4,69006430,2021-09-01 00:01:17.920,2096113,<p>You could do something like this if the str...,2013-02-21 15:43:28.953
...,...,...,...,...,...
3102831,77019850,2023-08-31 23:56:07.187,313768,<p>Make an inner solution routine that accepts...,2010-04-11 04:47:56.083
3102832,77019851,2023-08-31 23:56:19.123,13296852,<p>When you pass <code>stack[100]</code> as an...,2020-04-12 21:26:44.880
3102833,77019853,2023-08-31 23:56:47.647,5103949,"<p>I had the same error, I fixed it by followi...",2015-07-10 17:53:49.643
3102834,77019855,2023-08-31 23:57:36.797,16844882,"<p>In your Razor component, you can inject the...",2021-09-06 16:29:07.503


### Expand the new users' range to 24 months difference. (2 years to cover new users being in all month.)

In [18]:
# Compute difference between post creation_date - user creation_date.
ans_df['month_diff'] = (pd.to_datetime(ans_df['creation_date']) - pd.to_datetime(ans_df['account_creation_date'])) / pd.Timedelta(days=30)
## Define isNew variable
# set threshold as 12 months. (1 year)
ans_df['isNew'] = ans_df['month_diff'].apply(lambda x: 1 if x <= 24 else 0) # change from 12 to 24.
ans_df['year_month_day'] = pd.to_datetime(ans_df['creation_date']).dt.to_period('D').astype(str)
ans_df = ans_df[ans_df['month_diff'] >= 0]
ans_df

,id,creation_date,owner_user_id,body,account_creation_date,month_diff,isNew,year_month_day
0,69006420,2021-09-01 00:00:18.070,7332046,"<p>The short approach, remove python3.9 from y...",2016-12-22 19:33:51.100,57.106168,0,2021-09-01
1,69006421,2021-09-01 00:00:28.823,14471093,"<p>open the terminal, run as administrator.</p...",2020-10-18 06:29:51.557,10.590987,1,2021-09-01
2,69006422,2021-09-01 00:00:35.537,523612,"<p>Conceptually, <code>Base</code> has no reas...",2010-11-29 07:44:17.673,130.955933,0,2021-09-01
3,69006429,2021-09-01 00:01:09.140,11269158,<p><code>calldata</code> is a special data loc...,2019-03-27 23:15:22.047,29.601060,0,2021-09-01
4,69006430,2021-09-01 00:01:17.920,2096113,<p>You could do something like this if the str...,2013-02-21 15:43:28.953,103.778190,0,2021-09-01
...,...,...,...,...,...,...,...,...
3102831,77019850,2023-08-31 23:56:07.187,313768,<p>Make an inner solution routine that accepts...,2010-04-11 04:47:56.083,163.026578,0,2023-08-31
3102832,77019851,2023-08-31 23:56:19.123,13296852,<p>When you pass <code>stack[100]</code> as an...,2020-04-12 21:26:44.880,41.203462,0,2023-08-31
3102833,77019853,2023-08-31 23:56:47.647,5103949,"<p>I had the same error, I fixed it by followi...",2015-07-10 17:53:49.643,99.141735,0,2023-08-31
3102834,77019855,2023-08-31 23:57:36.797,16844882,"<p>In your Razor component, you can inject the...",2021-09-06 16:29:07.503,24.143715,0,2023-08-31


In [19]:
df_old = ans_df[ans_df['isNew'] == 0]
df_new = ans_df[ans_df['isNew'] == 1]
print("length of old_df is: ", len(df_old))
print("length of new_df is: ", len(df_new))

length of old_df is:  2242527
length of new_df is:  860289


### Preprocess

In [20]:
# erase NAs
df_old = df_old.dropna(subset=['owner_user_id'])
# Convert types
df_old['owner_user_id'] = df_old['owner_user_id'].astype(int).astype(str)
df_old['creation_date'] = pd.to_datetime(df_old['creation_date'])

# add year_month variable
df_old['year_month'] = df_old['creation_date'].dt.to_period('M')

# Get the total number of unique months
total_months = df_old['year_month'].nunique()
# Count the number of unique months for each user
user_month_counts = df_old.groupby('owner_user_id')['year_month'].nunique()
# Filter users who have written posts in every single month
users_in_every_month = user_month_counts[user_month_counts == total_months].index.tolist()

# Filtered data
df_filtered = df_old[df_old['owner_user_id'].isin(users_in_every_month)]
df_filtered

,id,creation_date,owner_user_id,body,account_creation_date,month_diff,isNew,year_month_day,year_month
6,69006435,2021-09-01 00:02:37.350,869736,<p>The error is correct. <code>getInputStream...,2011-07-29 16:32:37.833,122.877083,0,2021-09-01,2021-09
8,69006438,2021-09-01 00:03:36.527,8690857,<p>You should not use <code>.map</code> for si...,2017-09-28 15:59:11.263,47.777880,0,2021-09-01,2021-09
12,69006446,2021-09-01 00:04:33.053,16406,<p>The reason you need line B is called <a hre...,2008-09-17 16:54:47.547,157.709948,0,2021-09-01,2021-09
16,69006451,2021-09-01 00:05:09.193,478884,<p>For example you could do it this way:</p>\n...,2010-10-18 01:02:40.513,132.365335,0,2021-09-01,2021-09
48,69006529,2021-09-01 00:21:00.167,10527,<p>This is a REALLY good example as to why you...,2008-09-15 23:43:46.410,157.767528,0,2021-09-01,2021-09
...,...,...,...,...,...,...,...,...,...
3102806,77019800,2023-08-31 23:36:49.467,2156621,<p><code>StreamZip</code> + <code>expand</code...,2013-03-11 11:39:07.450,127.516613,0,2023-08-31,2023-08
3102818,77019822,2023-08-31 23:44:48.760,209103,"<p>The <a href=""https://firebase.google.com/do...",2009-11-11 22:30:27.020,168.035055,0,2023-08-31,2023-08
3102820,77019826,2023-08-31 23:46:31.700,939860,<p>You don't need a &quot;common key&quot; for...,2011-09-12 04:04:08.520,145.727370,0,2023-08-31,2023-08
3102823,77019833,2023-08-31 23:49:08.340,589924,<p>The <code>sprintf</code> operator behaves l...,2011-01-26 00:02:36.877,153.366355,0,2023-08-31,2023-08


In [21]:
# erase NAs
df_new = df_new.dropna(subset=['owner_user_id'])
# Convert types
df_new['owner_user_id'] = df_new['owner_user_id'].astype(int).astype(str)
df_new['creation_date'] = pd.to_datetime(df_new['creation_date'])

# add year_month variable
df_new['year_month'] = df_new['creation_date'].dt.to_period('M')

# Get the total number of unique months
total_months = df_new['year_month'].nunique()
# Count the number of unique months for each user
user_month_counts = df_new.groupby('owner_user_id')['year_month'].nunique()
# Filter users who have written posts in every single month
users_in_every_month = user_month_counts[user_month_counts == total_months].index.tolist()

# Filtered data
df_filtered2 = df_new[df_new['owner_user_id'].isin(users_in_every_month)]
df_filtered2

,id,creation_date,owner_user_id,body,account_creation_date,month_diff,isNew,year_month_day,year_month
31736,69085799,2021-09-07 09:37:58.397,16764520,<p>Extern &quot;C&quot; int mysym（without __de...,2021-08-27 05:36:48.330,0.372249,1,2021-09-07,2021-09
49111,69128903,2021-09-10 07:47:25.110,16764520,<p>I tried code: std::vector xLayerZero = {1} ...,2021-08-27 05:36:48.330,0.469690,1,2021-09-10,2021-09
79004,69204170,2021-09-16 07:36:25.057,16764520,<p>Try to use #pragma omp parallel num_threads...,2021-08-27 05:36:48.330,0.669435,1,2021-09-16,2021-09
96761,69248626,2021-09-20 02:49:31.247,16764520,"<p>To use RtMidi Classes, you need to follow ...",2021-08-27 05:36:48.330,0.796128,1,2021-09-20,2021-09
97688,69250974,2021-09-20 08:03:59.620,16764520,<p>You can try to use ref class. Please refer ...,2021-08-27 05:36:48.330,0.803407,1,2021-09-20,2021-09
...,...,...,...,...,...,...,...,...,...
3072761,76943862,2023-08-21 09:08:06.363,17026934,<p>[Disclaimer: I can't get CLSQL to work and ...,2021-09-28 15:40:12.457,23.057590,1,2023-08-21,2023-08
3073395,76945465,2023-08-21 12:51:20.290,17026934,"<p>With your definitions, <code>(make-city 'Be...",2021-09-28 15:40:12.457,23.062758,1,2023-08-21,2023-08
3089446,76986528,2023-08-27 09:52:20.787,17026934,<p>This answer deals largely with the example ...,2021-09-28 15:40:12.457,23.258614,1,2023-08-27,2023-08
3095375,77001375,2023-08-29 15:03:07.463,16895625,"<p>Based on this documentation on <a href=""htt...",2021-09-13 02:34:52.423,23.850654,1,2023-08-29,2023-08


### LSM

In [22]:
# old db
df_analysis_old = df_filtered.groupby(['owner_user_id', 'year_month'])['body'].agg(lambda x: ', '.join(map(str,x))).reset_index()
df_analysis_old

,owner_user_id,year_month,body
0,10008173,2021-09,<p>Since you already know the location on the ...
1,10008173,2021-10,<p>That <code>ENTRYPOINT</code> line doesn't f...
2,10008173,2021-11,<p>A Docker container only runs one process. ...
3,10008173,2021-12,<p>Connections between containers (over the Do...
4,10008173,2022-01,<p>You can use the Docker image name (<code>do...
...,...,...,...
17731,997358,2023-04,<p>To avoid various inefficiencies in your pro...
17732,997358,2023-05,<p>If you want an expression which is guarante...
17733,997358,2023-06,<p>How to optimize? Since you don't seem to h...
17734,997358,2023-07,"<p>Since the <a href=""/questions/tagged/jq"" cl..."


In [23]:
# new db
df_analysis_new = df_filtered2.groupby(['owner_user_id', 'year_month'])['body'].agg(lambda x: ', '.join(map(str,x))).reset_index()
df_analysis_new

,owner_user_id,year_month,body
0,16764520,2021-09,<p>Extern &quot;C&quot; int mysym（without __de...
1,16764520,2021-10,<pre><code>// decimal to hexadecimal conversio...
2,16764520,2021-11,<blockquote>\n<p>I've thought of some solution...
3,16764520,2021-12,<p><code>myIp(); ~myIp();</code> lack definiti...
4,16764520,2022-01,<p>This is the option I found in Visual Studio...
...,...,...,...
67,17026934,2023-04,<p>This is a bug in <code>iterate</code> which...
68,17026934,2023-05,"<p>Tim Bradshaw's <a href=""https://tfeb.github..."
69,17026934,2023-06,"<p>It uses <a href=""https://edicl.github.io/hu..."
70,17026934,2023-07,<p>To elaborate on the other answer. If you s...


In [24]:
print("Number of unique users in old db: ", df_analysis_old.owner_user_id.nunique())
print("Number of unique users in new db: ", df_analysis_new.owner_user_id.nunique())

Number of unique users in old db:  739
Number of unique users in new db:  3


In [26]:
# old
# Save individual body data into separate md file.
year_month = list(df_analysis_old['year_month'].unique().astype('str')) # get unique year_month
for i in range(len(year_month)):
    output_directory = f'/data1/StackOverflow/_Robustness/H4/lsm_data_fix/oldbie/{year_month[i]}'
    os.makedirs(output_directory, exist_ok=True)
    data = df_analysis_old[df_analysis_old['year_month'] == year_month[i]]
    for index, row in data.iterrows():
        user_filename = f"{output_directory}/user_{row['owner_user_id']}.md"
        with open(user_filename, 'w') as md_file:
            md_file.write(f"## User {row['owner_user_id']}\n")
            md_file.write(row['body'] + '\n\n')
    print(f"Data has been saved to individual md files in the '{output_directory}' directory.")

Data has been saved to individual md files in the '/data1/StackOverflow/_Robustness/H4/lsm_data_fix/oldbie/2021-09' directory.
Data has been saved to individual md files in the '/data1/StackOverflow/_Robustness/H4/lsm_data_fix/oldbie/2021-10' directory.


Data has been saved to individual md files in the '/data1/StackOverflow/_Robustness/H4/lsm_data_fix/oldbie/2021-11' directory.
Data has been saved to individual md files in the '/data1/StackOverflow/_Robustness/H4/lsm_data_fix/oldbie/2021-12' directory.
Data has been saved to individual md files in the '/data1/StackOverflow/_Robustness/H4/lsm_data_fix/oldbie/2022-01' directory.
Data has been saved to individual md files in the '/data1/StackOverflow/_Robustness/H4/lsm_data_fix/oldbie/2022-02' directory.
Data has been saved to individual md files in the '/data1/StackOverflow/_Robustness/H4/lsm_data_fix/oldbie/2022-03' directory.
Data has been saved to individual md files in the '/data1/StackOverflow/_Robustness/H4/lsm_data_fix/oldbie/2022-04' directory.
Data has been saved to individual md files in the '/data1/StackOverflow/_Robustness/H4/lsm_data_fix/oldbie/2022-05' directory.
Data has been saved to individual md files in the '/data1/StackOverflow/_Robustness/H4/lsm_data_fix/oldbie/2022

In [27]:
# New
# Save individual body data into separate md file.
year_month = list(df_analysis_new['year_month'].unique().astype('str')) # get unique year_month
for i in range(len(year_month)):
    output_directory = f'/data1/StackOverflow/_Robustness/H4/lsm_data_fix/newbie/{year_month[i]}'
    os.makedirs(output_directory, exist_ok=True)
    data = df_analysis_new[df_analysis_new['year_month'] == year_month[i]]
    for index, row in data.iterrows():
        user_filename = f"{output_directory}/user_{row['owner_user_id']}.md"
        with open(user_filename, 'w') as md_file:
            md_file.write(f"## User {row['owner_user_id']}\n")
            md_file.write(row['body'] + '\n\n')
    print(f"Data has been saved to individual md files in the '{output_directory}' directory.")

Data has been saved to individual md files in the '/data1/StackOverflow/_Robustness/H4/lsm_data_fix/newbie/2021-09' directory.
Data has been saved to individual md files in the '/data1/StackOverflow/_Robustness/H4/lsm_data_fix/newbie/2021-10' directory.
Data has been saved to individual md files in the '/data1/StackOverflow/_Robustness/H4/lsm_data_fix/newbie/2021-11' directory.
Data has been saved to individual md files in the '/data1/StackOverflow/_Robustness/H4/lsm_data_fix/newbie/2021-12' directory.
Data has been saved to individual md files in the '/data1/StackOverflow/_Robustness/H4/lsm_data_fix/newbie/2022-01' directory.
Data has been saved to individual md files in the '/data1/StackOverflow/_Robustness/H4/lsm_data_fix/newbie/2022-02' directory.
Data has been saved to individual md files in the '/data1/StackOverflow/_Robustness/H4/lsm_data_fix/newbie/2022-03' directory.
Data has been saved to individual md files in the '/data1/StackOverflow/_Robustness/H4/lsm_data_fix/newbie/2022

### Here, use separate py file for computing LSM
- compute_fix.py for ob
- compute2_fix.py for nb

### 1) Newbie

In [29]:
# Newbie: LSM Score Aggregation
import os
import pandas as pd
folder_path = '/data1/StackOverflow/_Robustness/H4/lsm_result_fix/newbie'
# Get a list of all files in the folder
file_list = os.listdir(folder_path)
file_list = sorted(file_list)
# Filter only pickle files
pickle_files = [file for file in file_list if file.endswith('.pickle')]

# Iterate through each pickle file and load it into a separate object
loaded_objects = pd.DataFrame(columns = ['User1', 'year_month', 'Similarity_toAvg'])
for pickle_file in pickle_files:
    file_path = os.path.join(folder_path, pickle_file)
    file_name = os.path.splitext(pickle_file)[0]
    # AGGREGATION
    with open(file_path, 'rb') as f:
        loaded_object = pickle.load(f)
    loaded_object['Similarity_toAvg'] = loaded_object['Similarity_toAvg'].astype(float)
    loaded_object = loaded_object.groupby('User1')['Similarity_toAvg'].mean().reset_index()
    loaded_object['year_month'] = file_name # Add year_month
    loaded_object = loaded_object[['User1', 'year_month', 'Similarity_toAvg']]
    # APPEND
    loaded_objects = pd.concat([loaded_objects, loaded_object])
    
loaded_objects = loaded_objects.rename(columns = {"User1": "owner_user_id"})
loaded_objects['owner_user_id'] = loaded_objects['owner_user_id'].apply(lambda x: x.replace('.md', ''))
loaded_objects['owner_user_id'] = loaded_objects['owner_user_id'].apply(lambda x: x.replace('user_', ''))    

/tmp/ipykernel_2086148/2543317767.py:24: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  loaded_objects = pd.concat([loaded_objects, loaded_object])


In [34]:
len(loaded_objects)

48

In [37]:
# Add T_d (Treated: 2022-09 ~ 2023-08 (12 months) Cntrol: 2021-09 ~ 2022-08 (12 months) )
import numpy as np
T_d = np.zeros(len(loaded_objects), dtype = int)
T_d[24:] = 1 # need to change.
loaded_objects['T_d'] = T_d

# Make month variable
loaded_objects['month'] = loaded_objects['year_month'].str[5:].astype(int).astype(str)
# Add P_t (if  9, 10, 11 month == P_t == 0, the others P_t == 1)
loaded_objects['P_t'] = loaded_objects['month'].apply(lambda x: 0 if x in ['9', '10', '11'] else 1)
loaded_objects = loaded_objects.rename(columns={'Similarity_toAvg': 'LSM'})
loaded_objects['ln_LSM'] = np.log(loaded_objects['LSM'])
loaded_objects

,owner_user_id,year_month,LSM,T_d,month,P_t,ln_LSM
0,16764520,2021-09,0.178735,0,9,0,-1.721854
1,16895625,2021-09,0.175238,0,9,0,-1.741611
0,16764520,2021-10,0.196717,0,10,0,-1.625989
1,16895625,2021-10,0.182847,0,10,0,-1.699104
0,16764520,2021-11,0.195968,0,11,0,-1.629806
1,16895625,2021-11,0.189451,0,11,0,-1.663623
0,16764520,2021-12,0.270331,0,12,1,-1.308107
1,16895625,2021-12,0.183957,0,12,1,-1.693052
0,16764520,2022-01,0.204997,0,1,1,-1.584759
1,16895625,2022-01,0.205084,0,1,1,-1.584337


In [40]:
loaded_objects = loaded_objects.reset_index(drop = True)

In [41]:
loaded_objects.describe()

,LSM,T_d,P_t,ln_LSM
count,48.000000,48.000000,48.000000,48.000000
mean,0.190283,0.500000,0.750000,-1.662707
std,0.017249,0.505291,0.437595,0.081195
min,0.172866,0.000000,0.000000,-1.755240
25%,0.180779,0.000000,0.750000,-1.710493
50%,0.187023,0.500000,1.000000,-1.676526
75%,0.195050,1.000000,1.000000,-1.634498
max,0.270331,1.000000,1.000000,-1.308107


In [42]:
# Save data
loaded_objects.to_csv('lsm_new_fix.csv', index = False)

### 2) Oldbie

In [1]:
# Oldbie: LSM Score Aggregation
import os
import pandas as pd
import pickle
folder_path = '/data1/StackOverflow/_Robustness/H4/lsm_result_fix/oldbie'
# Get a list of all files in the folder
file_list = os.listdir(folder_path)
file_list = sorted(file_list)
# Filter only pickle files
pickle_files = [file for file in file_list if file.endswith('.pickle')]

# Iterate through each pickle file and load it into a separate object
loaded_objects = pd.DataFrame(columns = ['User1', 'year_month', 'Similarity_toAvg'])
for pickle_file in pickle_files:
    file_path = os.path.join(folder_path, pickle_file)
    file_name = os.path.splitext(pickle_file)[0]
    # AGGREGATION
    with open(file_path, 'rb') as f:
        loaded_object = pickle.load(f)
    loaded_object['Similarity_toAvg'] = loaded_object['Similarity_toAvg'].astype(float)
    loaded_object = loaded_object.groupby('User1')['Similarity_toAvg'].mean().reset_index()
    loaded_object['year_month'] = file_name # Add year_month
    loaded_object = loaded_object[['User1', 'year_month', 'Similarity_toAvg']]
    # APPEND
    loaded_objects = pd.concat([loaded_objects, loaded_object])

loaded_objects = loaded_objects.rename(columns = {"User1": "owner_user_id"})
loaded_objects['owner_user_id'] = loaded_objects['owner_user_id'].apply(lambda x: x.replace('.md', ''))
loaded_objects['owner_user_id'] = loaded_objects['owner_user_id'].apply(lambda x: x.replace('user_', ''))

/tmp/ipykernel_654113/916238766.py:25: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  loaded_objects = pd.concat([loaded_objects, loaded_object])


In [2]:
len(loaded_objects)

17712

In [7]:
# Add T_d (Treated: 2022-09 ~ 2023-08 (12 months) Cntrol: 2021-09 ~ 2022-08 (12 months) )
import numpy as np
T_d = np.zeros(len(loaded_objects), dtype = int)

T_d[8856:] = 1 ### CHANGE THIS

loaded_objects['T_d'] = T_d

# Make month variable
loaded_objects['month'] = loaded_objects['year_month'].str[5:].astype(int).astype(str)
# Add P_t (if  9, 10, 11 month == P_t == 0, the others P_t == 1)
loaded_objects['P_t'] = loaded_objects['month'].apply(lambda x: 0 if x in ['9', '10', '11'] else 1)
loaded_objects = loaded_objects.rename(columns={'Similarity_toAvg': 'LSM'})
loaded_objects['ln_LSM'] = np.log(loaded_objects['LSM'])
loaded_objects = loaded_objects.reset_index(drop = True)
loaded_objects

,owner_user_id,year_month,LSM,T_d,month,P_t,ln_LSM
0,10008173,2021-09,0.179866,0,9,0,-1.715541
1,10112124,2021-09,0.189139,0,9,0,-1.665272
2,10138734,2021-09,0.176504,0,9,0,-1.734410
3,10140124,2021-09,0.170574,0,9,0,-1.768587
4,10157127,2021-09,0.181239,0,9,0,-1.707938
...,...,...,...,...,...,...,...
17707,9952196,2023-08,0.170362,1,8,1,-1.769828
17708,9957710,2023-08,0.168175,1,8,1,-1.782750
17709,99692,2023-08,0.172603,1,8,1,-1.756764
17710,9971759,2023-08,0.170889,1,8,1,-1.766741


In [8]:
loaded_objects.describe()

,LSM,T_d,P_t,ln_LSM
count,17712.000000,17712.000000,17712.000000,17712.000000
mean,0.185717,0.500000,0.750000,-1.691990
std,0.028668,0.500014,0.433025,0.120383
min,0.164936,0.000000,0.000000,-1.802199
25%,0.171959,0.000000,0.750000,-1.760501
50%,0.178193,0.500000,1.000000,-1.724889
75%,0.188757,1.000000,1.000000,-1.667296
max,0.527083,1.000000,1.000000,-0.640397


In [9]:
# Save data
loaded_objects.to_csv('lsm_old_fix.csv', index = False)